# FFF Workshop

## A6: Placing / Docking molecules

### Outline

- Fragmenstein placement

## Fragmenstein placement

Fragmenstein placement can minimise molecules into a reference protein structure with constraints to other ligands which share some (not necessarily exact) substructures.

A Squonk job is available for this that can be used to place ligands. As always some tokens are needed:

In [ ]:
token = "td09tr ... roprxi4"
squonk_token = "eyJhbGciOiJSU ... mKq_0E5g"

Placement tasks will need to specified using a list of dictionaries in the following format:

In [ ]:
placements = [
    dict(
        smiles="O=C1NC[C@@H](C(=O)NCc2cncnc2)N1", 
        inspirations=['A1081b', 'A1146a'],
        protein="A1081b",
    ),
    dict(
        smiles="O=C1NC[C@@H](C(=O)NCc2cncnc2)N1", 
        inspirations=['A1081b', 'A1146a'],
        protein="A1146a",
    )
]

Then a squonk job can be submitted:

In [ ]:
from fragalysis.requests import fragmenstein_place
tasks = fragmenstein_place(
    placements=placements,
    target_name="A71EV2A",
    tas="lb32627-66",
    stack="staging",
    token=token,
    num_repeats=1,
)
[task["squonk_instance_url"] for task in tasks]

N.B. Multiple jobs are created in groups of (reference, inspirations) as each individual fragmenstein job places against a single protein and single set of reference ligands.

Monitor the jobs until they complete:

In [ ]:
from fragalysis.requests.squonk import monitor_jobs
monitor_jobs(token=squonk_token, instance_ids=[task["squonk_instance"] for task in tasks])

And then fetch all the results:

In [ ]:
from pathlib import Path
from fragalysis.requests.squonk import get_file

destination = Path("placement_output") # this can be customised if you want the files elsewhere

for task in tasks:
    get_file(
        instance=task["squonk_instance"],
        path=task["output_file"],
        token=squonk_token,
        destination=f"{destination}/", 
    )

And they can be loaded into HIPPO:

In [ ]:
# setup the animal
import hippo
animal = hippo.HIPPO(
    "A71EV2A_demo",
    "../data/A71EV2A.sqlite",
)

In [ ]:
from pathlib import Path

destination = Path("placement_output")

for path in destination.glob("*.sdf"):

    # since reference and inspirations columns are currently missing from the fragmenstein output:
    task = [task for task in tasks if task["output_file"].endswith(path.name)][0]
    inspirations = task["inspirations"]
    reference=animal.poses[path.name.split("_")[-1].removesuffix(".sdf")]
    
    animal.load_sdf(
        target="A71EV2A",
        path=path,
        compound_tags=["placement_examples"],
        pose_tags=["placement_examples", "fragmenstein_placed"],
        energy_score_col="Energy",
        distance_score_col="RMSD",
        inspirations=inspirations,
        reference=reference,
    )
